### mlp mixer

In [1]:
import timm
import torch.nn as nn

mixer = timm.create_model('mixer_b16_224', pretrained=True)
num_classes = 3  # 새로운 클래스 수
mixer.head = nn.Linear(mixer.head.in_features, num_classes)
mixer.eval()


c:\Users\chldy\anaconda3\envs\torch_test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MlpMixer(
  (stem): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (blocks): Sequential(
    (0): MixerBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_tokens): Mlp(
        (fc1): Linear(in_features=196, out_features=384, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=384, out_features=196, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_channels): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      

In [2]:
import torch
import torch.nn as nn
checkpoint = torch.load("C:/Users/chldy/Downloads/mlp_mixer_class_3_final_86.33.bin", map_location='cpu') # GPU 환경이 아닌 경우 'cpu'를 사용합니다.
mixer.load_state_dict(checkpoint)



<All keys matched successfully>

### vit 

In [3]:
import timm
vit = timm.create_model('vit_base_patch16_224', pretrained=True)

num_classes = 3  # 새로운 클래스 수
vit.head = nn.Linear(vit.head.in_features, num_classes)


In [4]:
import torch
import torch.nn as nn
checkpoint = torch.load("C:/Users/chldy/Downloads/vit_class_3_checkpoint.bin", map_location='cpu') # GPU 환경이 아닌 경우 'cpu'를 사용합니다.
vit.load_state_dict(checkpoint)

<All keys matched successfully>

In [5]:
# 이미지 전처리 함수 정의
from torchvision import transforms
from PIL import Image

preprocess = transforms.Compose([
    transforms.Resize(224),  # 모델에 맞는 크기로 조정
    transforms.CenterCrop(224),  # 중앙을 기준으로 자르기
    transforms.ToTensor(),  # Tensor로 변환
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 정규화
])

# 이미지 불러오기 및 전처리
image_path = "C:/Users/chldy/Downloads/aa.jpg"
input_image = Image.open(image_path)
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # 배치 차원을 추가

In [21]:
import torch.nn.functional as F


with torch.no_grad():
    # MLP-Mixer 예측
    mlp_output = mixer(input_batch)
    mlp_probs = F.softmax(mlp_output, dim=1)
    
    # ViT 예측
    vit_output = vit(input_batch)
    vit_probs = F.softmax(vit_output, dim=1)
    
    # 평균 앙상블
    avg_probs = (mlp_probs + vit_probs) / 2
    final_prediction = avg_probs.argmax(dim=1).item()

# 각각의 모델 예측 확률 및 최종 예측 확률 출력
print("MLP-Mixer probabilities:", mlp_probs.numpy())
print("ViT probabilities:", vit_probs.numpy())
print("Ensemble probabilities:", avg_probs.numpy())
print("Final prediction:", final_prediction)

MLP-Mixer probabilities: [[0.33917397 0.34182787 0.31899816]]
ViT probabilities: [[9.9911350e-01 8.5955817e-04 2.6956297e-05]]
Ensemble probabilities: [[0.66914374 0.17134371 0.15951256]]
Final prediction: 0


### 배치 데이터 셋으로 f1 score 계산


In [6]:
import logging
import os
import json
from PIL import Image
from torchvision.transforms import ToTensor
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, RandomSampler, DistributedSampler, SequentialSampler
import torch
from sklearn.metrics import f1_score
import torch.nn.functional as F

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir,annotations_dir,  transform=None):
        """
        annotation_dir (string): 메타데이터가 있는 JSON 파일의 경로
        img_dir (string): 모든 이미지가 있는 디렉토리의 경로
        transform (callable, optional): 샘플에 적용될 선택적 변환
        """
        self.img_dir = img_dir
        self.transform = transform
        
        self.annotation_dir= annotations_dir
        

    def __len__(self):
        label_list= os.listdir(self.annotation_dir)
        return len(label_list)

    def __getitem__(self, idx):
        
        label_list= os.listdir(self.annotation_dir)
        
        img_path = os.path.join(self.img_dir, label_list[idx].split('.')[0]+'.'+label_list[idx].split('.')[1])

        try:
            image = Image.open(img_path)
        except (IOError, OSError) as e:
            print(f"Error loading image {img_path}: {e}")
            return self.__getitem__((idx + 1) % len(self))
        
        
        
        # faceExp_uploader 부분만 라벨로 사용
        with open(self.annotation_dir+'/'+label_list[idx],'r', encoding='utf-8') as f:
            self.image_labels=json.load(f)
        label = self.image_labels['faceExp_uploader']
        label_to_int = {'기쁨': 0, '당황': 1, '중립': 2}

        # 문자열 라벨을 정수로 매핑
        label_int = label_to_int[label]
        label_tensor = torch.tensor(label_int, dtype=torch.long)
        
        if self.transform:
            image_tensor = self.transform(image)
        else:
            # 기본적으로 이미지를 Tensor로 변환
            transform = ToTensor()
            image_tensor = transform(image)
        
        return image_tensor, label_tensor

In [7]:
def get_loader(img_size,  test_img_dir, test_annotation_dir, eval_batch_size):

    
    transform_test = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])

    
    testset = CustomDataset(img_dir=test_img_dir,
                            annotations_dir=test_annotation_dir,
                            transform=transform_test)
    

    test_sampler = SequentialSampler(testset) # 데이터셋에서 순차적으로 샘플을 선택, 데이터를 처음부터 끝까지 순서대로 샘플링
    
    test_loader = DataLoader(testset,
                             sampler=test_sampler,
                             batch_size=eval_batch_size,
                             num_workers=0,
                             pin_memory=True) if testset is not None else None

    return  test_loader

In [8]:
transform_test = transforms.Compose([
    transforms.Resize(224),  # 모델에 맞는 크기로 조정
    transforms.ToTensor(),  # Tensor로 변환
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 정규화
])

In [9]:
# 테스트 데이터셋 및 데이터로더 설정
test_img_dir = "C:/Users/chldy/Downloads/test_img_2030/test_img_2030"
test_annotation_dir = "C:/Users/chldy/Downloads/test_label_2030/test_label_2030"

In [11]:
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mixer.to(device)  # mixer 모델을 GPU로 이동
vit.to(device)  # vit 모델을 GPU로 이동

eval_batch_size = 32

test_loader= get_loader(224, test_img_dir, test_annotation_dir, eval_batch_size)


# 평가 모드로 전환
mixer.eval()
vit.eval()

all_labels = []
all_predictions = []

with torch.no_grad():
    for input_batch, labels in tqdm(test_loader, desc="Evaluating"):  # tqdm을 사용하여 진행 상태 표시
        input_batch, labels = input_batch.to(device), labels.to(device)  # 데이터를 GPU로 이동
        
        # 모델 예측
        mlp_output = mixer(input_batch)
        mlp_probs = F.softmax(mlp_output, dim=1)
        
        vit_output = vit(input_batch)
        vit_probs = F.softmax(vit_output, dim=1)
        
        # 평균 앙상블
        avg_probs = (mlp_probs + vit_probs) / 2
        predictions = avg_probs.argmax(dim=1)
        
        # 실제 라벨과 예측 라벨을 저장 (CPU로 이동하여 numpy 변환)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())

# F1 스코어 계산
f1 = f1_score(all_labels, all_predictions, average='weighted')
print(f"F1 Score: {f1:.4f}")

Evaluating:   0%|          | 0/57 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 57/57 [05:32<00:00,  5.84s/it]

F1 Score: 0.8920


: 

In [10]:
import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, recall_score, f1_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델을 GPU로 이동
mixer.to(device)
vit.to(device)

eval_batch_size = 32
test_loader = get_loader(224, test_img_dir, test_annotation_dir, eval_batch_size)

# 평가 모드로 전환
mixer.eval()
vit.eval()

# MLP Mixer 모델의 메트릭 계산
all_labels_mixer = []
all_predictions_mixer = []

with torch.no_grad():
    for input_batch, labels in tqdm(test_loader, desc="Evaluating MLP Mixer"):
        input_batch, labels = input_batch.to(device), labels.to(device)
        
        # 모델 예측
        mlp_output = mixer(input_batch)
        mlp_probs = F.softmax(mlp_output, dim=1)
        predictions = mlp_probs.argmax(dim=1)
        
        # 실제 라벨과 예측 라벨을 저장 (CPU로 이동하여 numpy 변환)
        all_labels_mixer.extend(labels.cpu().numpy())
        all_predictions_mixer.extend(predictions.cpu().numpy())

# 메트릭 계산
accuracy_mixer = accuracy_score(all_labels_mixer, all_predictions_mixer)
recall_mixer = recall_score(all_labels_mixer, all_predictions_mixer, average='weighted')
f1_mixer = f1_score(all_labels_mixer, all_predictions_mixer, average='weighted')
print(f"MLP Mixer - Accuracy: {accuracy_mixer:.4f}, Recall: {recall_mixer:.4f}, F1 Score: {f1_mixer:.4f}")

# ViT 모델의 메트릭 계산
all_labels_vit = []
all_predictions_vit = []

with torch.no_grad():
    for input_batch, labels in tqdm(test_loader, desc="Evaluating ViT"):
        input_batch, labels = input_batch.to(device), labels.to(device)
        
        # 모델 예측
        vit_output = vit(input_batch)
        vit_probs = F.softmax(vit_output, dim=1)
        predictions = vit_probs.argmax(dim=1)
        
        # 실제 라벨과 예측 라벨을 저장 (CPU로 이동하여 numpy 변환)
        all_labels_vit.extend(labels.cpu().numpy())
        all_predictions_vit.extend(predictions.cpu().numpy())

# 메트릭 계산
accuracy_vit = accuracy_score(all_labels_vit, all_predictions_vit)
recall_vit = recall_score(all_labels_vit, all_predictions_vit, average='weighted')
f1_vit = f1_score(all_labels_vit, all_predictions_vit, average='weighted')
print(f"ViT - Accuracy: {accuracy_vit:.4f}, Recall: {recall_vit:.4f}, F1 Score: {f1_vit:.4f}")


Evaluating MLP Mixer: 100%|██████████| 57/57 [05:33<00:00,  5.85s/it]


F1 Score for MLP Mixer: 0.8649


Evaluating ViT:   0%|          | 0/57 [00:00<?, ?it/s]c:\Users\chldy\anaconda3\envs\torch_test\lib\site-packages\timm\models\vision_transformer.py:92: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  x = F.scaled_dot_product_attention(
Evaluating ViT: 100%|██████████| 57/57 [05:35<00:00,  5.88s/it]

F1 Score for ViT: 0.8769
